# Building interactive Dashboards using Plotly's Dash

Dash is an open-source Python framework used for building analytical web applications. It is a powerful library that simplifies the development of data-driven applications. It’s especially useful for Python data scientists who aren’t very familiar with web development. Users can create amazing dashboards in their browser using dash.

Built on top of Plotly.js, React, and Flask, Dash ties modern UI elements like dropdowns, sliders and graphs directly to your analytical python code.

Dash apps consist of a Flask server that communicates with front-end React components using JSON packets over HTTP requests.

DASH APPLICATIONS ARE WRITTEN PURELY IN PYTHON, so NO HTML or JAVASCRIPT IS NECESSARY.

Following are some of the interactive dashboards built using Dash:

* [Visualize Earthquakes with Plotly Dash](https://belle-croissant-54211.herokuapp.com/)

* [Traffic Accidents UK ](https://traffic-accidents-uk.herokuapp.com/)

* [Uber Rides](https://dash-gallery.plotly.host/dash-uber-rides-demo/)

[Dash](https://dash.plotly.com/) official documentation can be visited to explore more.


Lets create a simple interactive dashboard and then break down all the elements in it.

## We always deal with datasets, so lets take a dataset and understand the different elements to create an interactive dashboard using Plotly's Dash

### Importing libraries

In [3]:
import dash
import dash_core_components as dcc
from jupyter_dash import JupyterDash
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

### Reading the data and creating a pivot table out of the same

In [4]:
df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/salesfunnel.xlsx?raw=True")
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [5]:
pv = pd.pivot_table(df, index=['Name'], columns=["Status"], values=['Quantity'], aggfunc=sum, fill_value=0)
pv.head()

Quantity                      
Status                       declined pending presented won
Name                                                       
Barton LLC                          1       0         0   0
Fritsch, Russel and Anderson        1       0         0   0
Herman LLC                          0       0         0   2
Jerde-Hilpert                       0       2         0   0
Kassulke, Ondricka and Metz         0       0         0   3

#### Now that the data is simply tabulated by customer and status, we can build out the bars that show the various customer status counts. 
#### The convention for plotly is that each item being plotted is usually called a trace . In this example, we want to plot Declined, Pending, Presented and Won as stacked bar charts so we need a trace for each one. Because we pivoted the data, the customer name is in the index and the individual values can be accessed through the hierarchical index on the column. Finally, the name will be used on the legend to describe the trace.

In [6]:
trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

#### Now that we have set everything up, we can build the actual dash app.

#### Below is the entire code, run the code :

In [13]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    html.H1(children='Sales Funnel Report'),
    html.Div(children='''National Sales Funnel Report.'''),
    dcc.Graph(
        id='example-graph',
        figure={
            'data': [trace1, trace2, trace3, trace4],
            'layout':
            go.Layout(title='Order Status by Customer', barmode='stack')
        })
])

app.run_server(mode = 'inline')

### Understanding the Dash app

#### The basic structure of this code is to build the app, layout the HTML components and create the figure dictionary which contains the instructions to build the graph as well as provide other layout options.

#### The most challenging aspect of this is the nesting of the HTML components. The fact that it was easy to get started without having to create the standard HTML/CSS/JS boilerplate needed for a modern web app. However, it is recommended  to use a good editor so it is easy to manage the code style and match the brackets and parentheses.

#### After laying out the HTML components, the next step is to define the figure using a dictionary that contains the figure as well as the data and layout options. In this case, the layout was needed to define that the bars were stacked and provide an overall title.

#### Once the app is laid out, we need to make sure it can run.

#### With just 30 to 40 lines of code, we get so much of interactivity. Amazing!

#### Pure PYTHON!




## Interactivity

#### The real power of Dash comes from its ability to do complex interactions.

Dash provides several interactive components out of the box including Dropdowns, Multi-Select Dropdowns, Radio Buttons, Checkboxes, Sliders, and Text Input. All of them can be easily constructed and tied into your plots to drive various interactive options.

We will take the above example a notch higher. For the more complex demo, we can modify the app to allow the user to select one of the managers via a dropdown and view the single manager’s results only.

In [8]:
# We get the unique managers here

mgr_options = df["Manager"].unique()

In [14]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2("Sales Funnel Report"),
    html.Div(
        [
            dcc.Dropdown(
                id="Manager",
                options=[{
                    'label': i,
                    'value': i
                } for i in mgr_options],
                value='All Managers'),
        ],
        style={'width': '25%',
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])


@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('Manager', 'value')])
def update_graph(Manager):
    if Manager == "All Managers":
        df_plot = df.copy()
    else:
        df_plot = df[df['Manager'] == Manager]

    pv = pd.pivot_table(
        df_plot,
        index=['Name'],
        columns=["Status"],
        values=['Quantity'],
        aggfunc=sum,
        fill_value=0)

    trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
    trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
    trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
    trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

    return {
        'data': [trace1, trace2, trace3, trace4],
        'layout':
        go.Layout(
            title='Customer Order Status for {}'.format(Manager),
            barmode='stack')
    }


app.run_server(mode='inline')

### Understanding the Dash app

### The next block of code will create the app and add the layout which will include a dcc.Dropdown to define the manager dropdown field.

```python
app = dash.Dash()

app.layout = html.Div([
    html.H2("Sales Funnel Report"),
    html.Div(
        [
            dcc.Dropdown(
                id="Manager",
                options=[{
                    'label': i,
                    'value': i
                } for i in mgr_options],
                value='All Managers'),
        ],
        style={'width': '25%',
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])
```


There a couple of items to note in this section. The dropdown needs to have an id defined that can be referenced later to drive our interactivity. You may also notice that the dcc.Graph code only contains an id and not the rest of the content we saw earlier. Since this is an interactive plot the dcc.Graph acts as a placeholder that we will define in a moment.

The final aspect of this section is that there is a style attribute which is a convenient way to make sure the component (in this case the dropdown) is only 25% of the screen and does not span the entire browser.

Now that the layout is completed, we can build up the interactive components by adding a callback decorator to a function that manipulates the data and returns a dictionary. This resulting dictionary looks like the figure dictionary defined in the simpler example above so the basic concept continues to build upon itself.



### Basic callback

```python
@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('Manager', 'value')])
```

This looks a little complex but all it is doing is providing a wrapper around the update_graph function that defines an output and an input. In this case the output is the funnel-graph and the input Manager. In other words this decorator knows how to link the drop down definition to the function and take the output and place it in the dcc.Graph . This little bit of decorator magic is where the real power of Dash can be seen. It may seem a little complex at first but the basic concept is straightforward and once you understand it, you can really build some complex interactive visualizations.

```python
def update_graph(Manager):
    if Manager == "All Managers":
        df_plot = df.copy()
    else:
        df_plot = df[df['Manager'] == Manager]

    pv = pd.pivot_table(
        df_plot,
        index=['Name'],
        columns=["Status"],
        values=['Quantity'],
        aggfunc=sum,
        fill_value=0)
```

This portion of the function filters the data based on the dropdown to include only a subset of the managers - or include all managers is the default is selected. After filtering the dataframe, the code pivots the data to construct the tabular results similar to the earlier example. Pure python

### The next portion of the code defines the traces, we have done this earlier:

```python
trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')
```

### The final step is to return a dictionary with the figure and layout defined. The decorator we defined earlier will make sure that the updated chart gets included in the output display.

```python
return {
    'data': [trace1, trace2, trace3, trace4],
    'layout':
    go.Layout(
        title='Customer Order Status for {}'.format(Manager),
        barmode='stack')
}
```

### Run the app



```python
app.run_server(mode='inline')
```

So, how does your app look?

Play around with it.

#### This application is about 68 lines of code which is not too bad when you consider how much is being done:

* Fully interactive interface with the charts
* Drop down selection of various managers
* Ability to save the plot as a png file
* Pan and zoom options
* Multiple level undo
* Automatic resizing of the plot as window sizes change


## Activity :

* What do you think is the role of reporting in Data Science?

* What are the other visualization tools you are familiar of?
